In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
# import XGBoost
from xgboost import XGBClassifier
import os
import itertools
import tqdm
logistic_regression_param_grid = {
            # 'penalty': ['l1', 'l2'],
            # 'solver' : ['liblinear'],
            'penalty': ['elasticnet'],
            'solver' : ['saga'],
            'l1_ratio': [0, 0.25, 0.33, 0.5, 0.66, 0.75, 1],
            'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10],
            'class_weight': ['balanced'],
            'max_iter': [10000],
            'tol': [1e-3, 1e-4],
            }

In [3]:
data_dir= '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_finetune_optimization/April_30_Finetune_Data'
local_dir = os.path.expanduser('~/Desktop/saved_models')



In [15]:
X_train = pd.read_csv(os.path.join(data_dir, 'X_finetune_train.csv'),index_col=0)
y_train = pd.read_csv(os.path.join(data_dir, 'y_finetune_train.csv'),index_col=0)

X_val = pd.read_csv(os.path.join(data_dir, 'X_finetune_val.csv'),index_col=0)
y_val = pd.read_csv(os.path.join(data_dir, 'y_finetune_val.csv'),index_col=0)

X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

X_test = pd.read_csv(os.path.join(data_dir, 'X_finetune_test.csv'),index_col=0)
y_test = pd.read_csv(os.path.join(data_dir, 'y_finetune_test.csv'),index_col=0)

In [16]:
y_col = 'IMDC BINARY'
y_train = y_train[y_col]
y_train.dropna(inplace=True)
X_train = X_train.loc[y_train.index]

y_val = y_val[y_col]
y_val.dropna(inplace=True)
X_val = X_val.loc[y_val.index]

y_trainval = y_trainval[y_col]
y_trainval.dropna(inplace=True)
X_trainval = X_trainval.loc[y_trainval.index]

y_test = y_test[y_col]
y_test.dropna(inplace=True)
X_test = X_test.loc[y_test.index]

In [12]:
y_val.shape

(61,)

In [11]:
X_val.shape

(61, 2736)

In [17]:
param_grid = logistic_regression_param_grid

param_combs = list(itertools.product(*param_grid.values()))
grid_search = [dict(zip(param_grid.keys(), values)) for values in param_combs]
hp_summary = {}
for i, gs in tqdm.tqdm(enumerate(grid_search)):

    model = LogisticRegression(**gs)
    model.fit(X_train.to_numpy(), y_train.to_numpy())
    y_pred = model.predict(X_val.to_numpy())
    y_pred_proba = model.predict_proba(X_val.to_numpy())[:,1]
    auc = roc_auc_score(y_val.to_numpy(), y_pred_proba)
    acc = accuracy_score(y_val.to_numpy(), y_pred)
    hp_summary[i] = {'auc': auc, 'acc': acc, 'params': gs}


140it [10:28,  4.49s/it]


In [19]:
import json

In [20]:
local_dir

'/Users/jonaheaton/Desktop/saved_models'

In [21]:
with open(os.path.join(local_dir, 'logistic_regression_hp_summary.json'), 'w') as f:
    json.dump(hp_summary, f)

In [22]:
# create a dataframe from the dictionary
hp_summary_df = pd.DataFrame.from_dict(hp_summary, orient='index')

In [23]:
hp_summary_df

,auc,acc,params
0,0.909091,0.786885,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."
1,0.909091,0.786885,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."
2,0.905594,0.803279,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."
3,0.905594,0.803279,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."
4,0.905594,0.803279,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."
...,...,...,...
135,0.870629,0.770492,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."
136,0.902098,0.819672,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."
137,0.874126,0.770492,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."
138,0.903263,0.836066,"{'penalty': 'elasticnet', 'solver': 'saga', 'l..."


In [24]:
hp_summary_df.to_csv(os.path.join(local_dir, 'logistic_regression_hp_summary.csv'))

In [26]:
# refit the top result from the hp summary

# best_hp = max(hp_summary, key=lambda x: hp_summary[x]['auc'])
# best_params = hp_summary[best_hp]['params']

best_hp = hp_summary_df['auc'].idxmax()
print(f'Best HP: {best_hp}')
best_params = hp_summary_df.loc[best_hp, 'params']
print(f'Best HP: {best_params}')
model = LogisticRegression(**best_params)

model.fit(X_trainval, y_trainval)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:,1]
y_trainval_pred_proba = model.predict_proba(X_trainval)[:,1]
auc = roc_auc_score(y_test, y_pred_proba)
auc_trainval = roc_auc_score(y_trainval, y_trainval_pred_proba)
acc = accuracy_score(y_test, y_pred)

print(f'Test AUC: {auc}')
print('TrainVal AUC: ', auc_trainval)





Best HP: 0
Best HP: {'penalty': 'elasticnet', 'solver': 'saga', 'l1_ratio': 0, 'C': 0.001, 'class_weight': 'balanced', 'max_iter': 10000, 'tol': 0.001}
Best HP: {'penalty': 'elasticnet', 'solver': 'saga', 'l1_ratio': 0, 'C': 0.001, 'class_weight': 'balanced', 'max_iter': 10000, 'tol': 0.001}
Test AUC: 0.9539641943734016
TrainVal AUC:  0.9912967032967033


In [29]:
y_col = 'MSKCC BINARY'


X_train = pd.read_csv(os.path.join(data_dir, 'X_finetune_train.csv'),index_col=0)
y_train = pd.read_csv(os.path.join(data_dir, 'y_finetune_train.csv'),index_col=0)

X_val = pd.read_csv(os.path.join(data_dir, 'X_finetune_val.csv'),index_col=0)
y_val = pd.read_csv(os.path.join(data_dir, 'y_finetune_val.csv'),index_col=0)

X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

X_test = pd.read_csv(os.path.join(data_dir, 'X_finetune_test.csv'),index_col=0)
y_test = pd.read_csv(os.path.join(data_dir, 'y_finetune_test.csv'),index_col=0)


y_train = y_train[y_col]
y_train.dropna(inplace=True)
X_train = X_train.loc[y_train.index]

y_val = y_val[y_col]
y_val.dropna(inplace=True)
X_val = X_val.loc[y_val.index]

y_trainval = y_trainval[y_col]
y_trainval.dropna(inplace=True)
X_trainval = X_trainval.loc[y_trainval.index]

y_test = y_test[y_col]
y_test.dropna(inplace=True)
X_test = X_test.loc[y_test.index]

In [30]:
from xgboost import XGBClassifier

# Instantiate the classifier
clf = XGBClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_val)
y_pred_proba = clf.predict_proba(X_val)[:,1]
auc = roc_auc_score(y_val, y_pred_proba)
acc = accuracy_score(y_val, y_pred)

y_train_pred_proba = clf.predict_proba(X_train)[:,1]
train_auc = roc_auc_score(y_train, y_train_pred_proba)

print('XGBoost')
print(f'Val AUC: {auc}')
# print(f'Train AUC: {train_auc}')

# print(f'Val Acc: {acc}')


# Instantiate the classifier
clf = XGBClassifier()

# Train the classifier
clf.fit(X_trainval, y_trainval)

# Predict the labels of the test set
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_pred_proba)
acc = accuracy_score(y_test, y_pred)

y_trainval_pred_proba = clf.predict_proba(X_trainval)[:,1]
train_auc = roc_auc_score(y_trainval, y_trainval_pred_proba)

print(f'Test AUC: {auc}')
# print(f'TrainVal AUC: {train_auc}')

# print(f'Val Acc: {acc}')


XGBoost
Val AUC: 0.830026455026455
Test AUC: 0.9087035909920876


In [31]:
# run random forest

from sklearn.ensemble import RandomForestClassifier

# Instantiate the classifier
clf = RandomForestClassifier()
print('Random Forest')

# Train the classifier
clf.fit(X_train, y_train)

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_val)
y_pred_proba = clf.predict_proba(X_val)[:,1]
auc = roc_auc_score(y_val, y_pred_proba)
acc = accuracy_score(y_val, y_pred)

y_train_pred_proba = clf.predict_proba(X_train)[:,1]
train_auc = roc_auc_score(y_train, y_train_pred_proba)


print(f'Val AUC: {auc}')
# print(f'Train AUC: {train_auc}')

# print(f'Val Acc: {acc}')

# Train the classifier
clf.fit(X_trainval, y_trainval)

# Predict the labels of the test set
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_pred_proba)
acc = accuracy_score(y_test, y_pred)

y_trainval_pred_proba = clf.predict_proba(X_trainval)[:,1]
train_auc = roc_auc_score(y_trainval, y_trainval_pred_proba)

print(f'Test AUC: {auc}')

Random Forest
Val AUC: 0.8783068783068783
Test AUC: 0.8706634205721242


In [32]:
# Run on SVC
from sklearn.svm import SVC

# Instantiate the classifier
clf = SVC(probability=True)
print('SVC')

# Train the classifier
clf.fit(X_train, y_train)

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_val)
y_pred_proba = clf.predict_proba(X_val)[:,1]
auc = roc_auc_score(y_val, y_pred_proba)
acc = accuracy_score(y_val, y_pred)

y_train_pred_proba = clf.predict_proba(X_train)[:,1]
train_auc = roc_auc_score(y_train, y_train_pred_proba)


print(f'Val AUC: {auc}')
# print(f'Train AUC: {train_auc}')

# print(f'Val Acc: {acc}')

# Train the classifier
clf.fit(X_trainval, y_trainval)

# Predict the labels of the test set
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_pred_proba)
acc = accuracy_score(y_test, y_pred)

y_trainval_pred_proba = clf.predict_proba(X_trainval)[:,1]
train_auc = roc_auc_score(y_trainval, y_trainval_pred_proba)

print(f'Test AUC: {auc}')

SVC
Val AUC: 0.8756613756613757
Test AUC: 0.9099208764455264


In [33]:
# logistic regression
from sklearn.linear_model import LogisticRegression

# Instantiate the classifier
clf = LogisticRegression()
print('Logistic Regression')

# Train the classifier
# Train the classifier
clf.fit(X_train, y_train)

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_val)
y_pred_proba = clf.predict_proba(X_val)[:,1]
auc = roc_auc_score(y_val, y_pred_proba)
acc = accuracy_score(y_val, y_pred)

y_train_pred_proba = clf.predict_proba(X_train)[:,1]
train_auc = roc_auc_score(y_train, y_train_pred_proba)


print(f'Val AUC: {auc}')
# print(f'Train AUC: {train_auc}')

# print(f'Val Acc: {acc}')

# Train the classifier
clf.fit(X_trainval, y_trainval)

# Predict the labels of the test set
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_pred_proba)
acc = accuracy_score(y_test, y_pred)

y_trainval_pred_proba = clf.predict_proba(X_trainval)[:,1]
train_auc = roc_auc_score(y_trainval, y_trainval_pred_proba)

print(f'Test AUC: {auc}')

Logistic Regression
Val AUC: 0.8630952380952381
Test AUC: 0.9099208764455265


In [34]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Instantiate the classifier
clf = DecisionTreeClassifier()
print('Decision Tree')

# Train the classifier
clf.fit(X_train, y_train)

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_val)
y_pred_proba = clf.predict_proba(X_val)[:,1]
auc = roc_auc_score(y_val, y_pred_proba)
acc = accuracy_score(y_val, y_pred)

y_train_pred_proba = clf.predict_proba(X_train)[:,1]
train_auc = roc_auc_score(y_train, y_train_pred_proba)


print(f'Val AUC: {auc}')
# print(f'Train AUC: {train_auc}')

# print(f'Val Acc: {acc}')

# Train the classifier
clf.fit(X_trainval, y_trainval)

# Predict the labels of the test set
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_pred_proba)
acc = accuracy_score(y_test, y_pred)

y_trainval_pred_proba = clf.predict_proba(X_trainval)[:,1]
train_auc = roc_auc_score(y_trainval, y_trainval_pred_proba)

print(f'Test AUC: {auc}')

Decision Tree
Val AUC: 0.7493386243386242
Test AUC: 0.7994522215459525
